In [ ]:
# utf-8
import pandas as pd
from fake_useragent import UserAgent
import requests,json
from lxml import etree
import os
import urllib.request

In [ ]:
base_url = 'https://www.cbndata.com/api/v2/report_products?tags[]=all&year=all&per=12&page='

In [ ]:
ua = UserAgent()
used_Agent = ua.ie

In [ ]:
# base_url+'1'
# used_Agent

In [ ]:
def get_url(ori_url =  base_url,page=1):
    real_url = ori_url+str(page)
#     r = requests.get(url = real_url,headers = used_Agent)
    r = requests.get(url = real_url)
    if r.status_code == 200:
        return r.text
    else:
        return r.text
    
def get_ori_url(ori_url =  base_url):
    r = requests.get(url = ori_url)
    if r.status_code == 200:
        return r.text
    else:
        return r.text

def get_pd(content):
    return pd.read_json(json.dumps(json.loads(content)['data']))

def get_all_pages(content):
    current_page = json.loads(content)['meta']['total_page']
    return current_page
    
def get_cur_pages(content):
    current_page = json.loads(content)['meta']['current_page']
    return current_page

def get_all_pds(base_url,start_num=1):
    content = get_url(base_url,start_num)
    max_pages = int(get_all_pages(content))
    current_page = int(get_cur_pages(content))
    print('matx_pages',max_pages)
    print('current_page',current_page)
    tem_pd = get_pd(content)
    base_pd = tem_pd
    for i in range(start_num+1,max_pages+1):
        print('current_page',i)
        content = get_url(base_url,i)
        tem_pd = get_pd(content)
        base_pd = pd.concat([base_pd,tem_pd],ignore_index=True)
    return base_pd

def get_detail_pages(report_id = 1898):
    return 'https://m.cbndata.com/report/%d/detail?isReading=report&page=1' %(report_id)

def  get_script(content):
    selector = etree.HTML(content)
    pic_url_json = selector.xpath('/html/body/script[1]/text()')[0].strip().strip('  window.__INITIAL_STATE__ = ').strip(';')
    pic_url_list = json.loads(pic_url_json)
    return pic_url_list

def write_json(data,edit_path):
    file_path = r"H:\data\Spider\data\sec\%s.json" %(edit_path)
    with open(file_path,'w') as f:
        json.dump(data,f)
        print(edit_path,'ending...')
        
def read_json(edit_path):
    with open(edit_path,'r') as f:
        data_json =json.load(f).get('report').get('watermark_image_urls')
        return data_json
    
def download_pic(img_url,img_name):
    headers = {
              'Content-Type': 'application/json;charset=utf-8',
              'User-Agent': 'Mozilla/5.0 (Linux; Android 8.0.0; Pixel 2 XL Build/OPD1.170816.004) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Mobile Safari/537.36',
              }
    img_url = img_url.replace('https','http').strip()
    urllib.request.urlretrieve(img_url,str(str(img_name)+'.jpg').strip())
    print(img_url)
    print(str(img_name)+'.jpg','图片下载完成')

In [ ]:
# 'https://cfpdf.dtcj.com/852-1559114874-02_watermarked.jpg?e=1568615447&token=9GfIMMn9rtLBXpIW4xLvmgwzCDAsMHSqzoJ5LdQe:0lKh9C9OPGSBF6oYyS8kY4NroO4='.replace('https','http')

# output

In [ ]:
# fir
# tem_da = get_all_pds(base_url)
# tem_da.to_excel(r'H:\data\Spider\data\zero\seed.xlsx')
tem_da = pd.read_excel(r'H:\data\Spider\data\zero\seed.xlsx')

In [ ]:
detail_ids = tem_da['id'].tolist()
tem_detail_urls = [_ for _ in map(get_detail_pages,detail_ids)]
# len(tem_detail_urls)

In [ ]:
# #  sec
# for _ in tem_detail_urls[0:len(tem_detail_urls)]:
# # for _ in tem_detail_urls[0:10]:
#     tem_content = get_ori_url(_)
#     script_json = get_script(tem_content)
#     write_json(script_json,script_json.get('title').replace('|',"")+'_'+str(script_json.get('id')))
# print('ending')

In [ ]:
# for _ in tem_detail_urls[0:len(tem_detail_urls)]:
for _ in tem_detail_urls[0:len(tem_detail_urls)]:
    tem_content = get_ori_url(_)
    script_json = get_script(tem_content)
    title = script_json.get('title').replace('|',"")+'_'+str(script_json.get('id'))
    if title == '“泛90后”男性男装消费偏好研究报告_337':
        try:
            tem_content = get_ori_url(_)
            script_json = get_script(tem_content)
            title = script_json.get('title').replace('|',"")+'_'+str(script_json.get('id'))
            pic_urls_down =script_json.get('report').get('watermark_image_urls')
            pic_file_dir  =  'H:\data\Spider\data\details\\'
            real_paths_down = os.path.join(pic_file_dir, title)
            if not os.path.exists(os.path.join(output_path, title)):
                os.makedirs(os.path.join(output_path, title))
            else:
                pass
            for index_i,pic_url in enumerate(pic_urls_down):
                print('pic_url',pic_url)
                print('real_paths',real_paths_down+'\\'+str(index_i)+'.jpg')
                download_pic(pic_url,real_paths_down+'\\'+str(index_i))
        except:
            continue
print('ending')

In [ ]:
# thr
all_paths = [os.path.join('H:\data\Spider\data\sec\\',_) for _ in os.listdir('H:\data\Spider\data\sec\\')]
all_names = [os.path.basename(_).split('.')[0] for _ in os.listdir('H:\data\Spider\data\sec\\')]
all_names.index('“泛90后”男性男装消费偏好研究报告_337')

In [10]:
output_path =  'H:\data\Spider\data\details\\'
for index,_ in enumerate(all_paths):
    if index==187:
        try:
            print(index)
            pic_urls_down = read_json(_)
            title = os.path.basename(_).split('.')[0]
            real_paths_down = os.path.join(output_path, title)
            if not os.path.exists(os.path.join(output_path, title)):
                os.makedirs(os.path.join(output_path, title))
                for index_i,pic_url in enumerate(pic_urls_down):
                    print('pic_url',pic_url)
                    print('real_paths',real_paths_down+'\\'+str(index_i)+'.jpg')
                    download_pic(pic_url,real_paths_down+'\\'+str(index_i))
            else:
                for index_i,pic_url in enumerate(pic_urls_down):
                    print('pic_url',pic_url)
                    print('real_paths',real_paths_down+'\\'+str(index_i)+'.jpg')
                    download_pic(pic_url,real_paths_down+'\\'+str(index_i))
        except:
            continue
    #          
    

187
pic_url https://cfpdf.dtcj.com/9fb6e197-d473-4c8e-b06c-0103a080f6e8/images/1_watermarked.jpg?e=1568621960&token=9GfIMMn9rtLBXpIW4xLvmgwzCDAsMHSqzoJ5LdQe:cnb5eMxiDtmvTgBJRZzZl74hhdI=
real_paths H:\data\Spider\data\details\“泛90后”男性男装消费偏好研究报告_337\0.jpg


In [8]:
# download_pic('http://cfpdf.dtcj.com/852-1559114874-05_watermarked.jpg?e=1568615447&token=9GfIMMn9rtLBXpIW4xLvmgwzCDAsMHSqzoJ5LdQe:TVSH_3FPW4pMWC2yUrK-wlDdnO0=',
#             'H:\data\Spider\data\details\鲍跃忠：新零售建立在清晰商业逻辑基础上的数据驱动_123\\4')

http://cfpdf.dtcj.com/852-1559114874-05_watermarked.jpg?e=1568615447&token=9GfIMMn9rtLBXpIW4xLvmgwzCDAsMHSqzoJ5LdQe:TVSH_3FPW4pMWC2yUrK-wlDdnO0=
H:\data\Spider\data\details\鲍跃忠：新零售建立在清晰商业逻辑基础上的数据驱动_123\4.jpg 图片下载完成


In [62]:
# #  sec
for _ in tem_detail_urls[0:len(tem_detail_urls)]:
    tem_content = get_ori_url(_)
    script_json = get_script(tem_content)
    write_json(script_json,script_json.get('title').replace('|',"")+'_'+str(script_json.get('id')))
print('ending')





In [14]:
x